In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [2]:
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

In [4]:
df = pd.read_csv("Crop_yield.csv")
df

State_Name  Crop_Year Crop_Name  Season        Area  \
0    Andaman and Nicobar Islands       2000      RICE  Kharif    10881.00   
1                 Andhra Pradesh       2000      RICE  Kharif  2565400.00   
2              Arunachal Pradesh       2000      RICE  Kharif   124545.74   
3                          Assam       2000      RICE  Kharif  2525653.00   
4                          Bihar       2000      RICE  Kharif   665467.00   
..                           ...        ...       ...     ...         ...   
491                       Sikkim       2015      RICE  Kharif    12144.00   
492                   Tamil Nadu       2015      RICE  Kharif  1493276.00   
493                Uttar Pradesh       2015      RICE  Kharif  5861282.00   
494                  Uttarakhand       2015      RICE  Kharif   292147.87   
495                  West Bengal       2015      RICE  Kharif  5711237.07   

      Production  Rainfall  Minimum Selling Price  Yield  
0       32184.00    1244.2                  510.0   2.96  
1     6436200.00     965.3                  510.0   2.51  
2      159860.30    1650.0                  510.0   1.28  
3     2255230.00    1666.6                  510.0   0.89  
4      945254.00    1047.7                  510.0   1.42  
..           ...       ...                    ...    ...  
491     21340.00    1883.0                 1410.0   1.76  
492   4050334.00     285.9                 1410.0   2.71  
493  14415939.00     436.1                 1410.0   2.46  
494    611450.53     881.5                 1410.0   2.09  
495  14357221.60    1304.9                 1410.0   2.51  

[496 rows x 9 columns]

In [5]:
df.rename(columns={"Yield": "yield"}, inplace=True)

In [6]:
df.fillna(df.mean(numeric_only=True), inplace=True)

In [7]:
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
categorical_cols

['State_Name', 'Crop_Name', 'Season']

In [8]:
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

In [9]:
joblib.dump(label_encoders, f"{models_dir}/label_encoders.pkl")

['models/label_encoders.pkl']

In [10]:
X = df.drop(columns=["yield"])
y = df["yield"]

In [12]:
X

State_Name  Crop_Year  Crop_Name  Season        Area   Production  \
0             0       2000          0       0    10881.00     32184.00   
1             1       2000          0       0  2565400.00   6436200.00   
2             2       2000          0       0   124545.74    159860.30   
3             3       2000          0       0  2525653.00   2255230.00   
4             4       2000          0       0   665467.00    945254.00   
..          ...        ...        ...     ...         ...          ...   
491          27       2015          0       0    12144.00     21340.00   
492          28       2015          0       0  1493276.00   4050334.00   
493          29       2015          0       0  5861282.00  14415939.00   
494          30       2015          0       0   292147.87    611450.53   
495          31       2015          0       0  5711237.07  14357221.60   

     Rainfall  Minimum Selling Price  
0      1244.2                  510.0  
1       965.3                  510.0  
2      1650.0                  510.0  
3      1666.6                  510.0  
4      1047.7                  510.0  
..        ...                    ...  
491    1883.0                 1410.0  
492     285.9                 1410.0  
493     436.1                 1410.0  
494     881.5                 1410.0  
495    1304.9                 1410.0  

[496 rows x 8 columns]

In [14]:
y

0      2.96
1      2.51
2      1.28
3      0.89
4      1.42
       ... 
491    1.76
492    2.71
493    2.46
494    2.09
495    2.51
Name: yield, Length: 496, dtype: float64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

In [17]:
best_model = None
best_rmse = float('inf')  
best_model_name = ""

In [22]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"🔹 {name} Performance:")
    print(f"   ✅ RMSE: {rmse:.4f}")
    print(f"   ✅ R² Score: {r2:.4f}")

    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_model_name = name

🔹 Linear Regression Performance:
   ✅ RMSE: 0.5550
   ✅ R² Score: 0.4691
🔹 Random Forest Performance:
   ✅ RMSE: 0.2535
   ✅ R² Score: 0.8893
🔹 Gradient Boosting Performance:
   ✅ RMSE: 0.3129
   ✅ R² Score: 0.8312


In [23]:
joblib.dump(best_model, f"{models_dir}/best_yield_prediction.pkl")

['models/best_yield_prediction.pkl']

In [25]:
print(f"Best Model: {best_model_name} with RMSE: {best_rmse:.4f}")

Best Model: Random Forest with RMSE: 0.2535
